In [ ]:
# import xarray as xr

# grib_file = "panguweather.grib"
# ds = xr.load_dataset(grib_file, engine="cfgrib")

skipping variable: paramId==167 shortName='t2m'
Traceback (most recent call last):
  File "/home/sylvain/Documents/training/ecmwf/.venv/lib/python3.11/site-packages/cfgrib/dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/sylvain/Documents/training/ecmwf/.venv/lib/python3.11/site-packages/cfgrib/dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='heightAboveGround' value=Variable(dimensions=(), data=np.float64(10.0)) new_value=Variable(dimensions=(), data=np.float64(2.0))


In [1]:
import xarray as xr

grib_file = "panguweather.grib"
ds = xr.open_dataset(grib_file, engine="cfgrib")

skipping variable: paramId==167 shortName='t2m'
Traceback (most recent call last):
  File "/home/sylvain/Documents/training/ecmwf/.venv/lib/python3.11/site-packages/cfgrib/dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/sylvain/Documents/training/ecmwf/.venv/lib/python3.11/site-packages/cfgrib/dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='heightAboveGround' value=Variable(dimensions=(), data=np.float64(10.0)) new_value=Variable(dimensions=(), data=np.float64(2.0))


In [5]:
ds

<xarray.Dataset> Size: 17GB
Dimensions:            (step: 61, isobaricInhPa: 13, latitude: 721,
                        longitude: 1440)
Coordinates:
    time               datetime64[ns] 8B ...
  * step               (step) timedelta64[ns] 488B 0 days 00:00:00 ... 15 day...
  * isobaricInhPa      (isobaricInhPa) float64 104B 1e+03 925.0 ... 100.0 50.0
  * latitude           (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude          (longitude) float64 12kB -180.0 -179.8 ... 179.5 179.8
    valid_time         (step) datetime64[ns] 488B ...
    meanSea            float64 8B ...
    heightAboveGround  float64 8B ...
Data variables:
    z                  (step, isobaricInhPa, latitude, longitude) float32 3GB ...
    q                  (step, isobaricInhPa, latitude, longitude) float32 3GB ...
    t                  (step, isobaricInhPa, latitude, longitude) float32 3GB ...
    u                  (step, isobaricInhPa, latitude, longitude) float32 3GB ...
    v                  (step, isobaricInhPa, latitude, longitude) float32 3GB ...
    msl                (step, latitude, longitude) float32 253MB ...
    u10                (step, latitude, longitude) float32 253MB ...
    v10                (step, latitude, longitude) float32 253MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-04-27T19:13 GRIB to CDM+CF via cfgrib-0.9.1...

In [2]:
output_dir = "maps"
fields = ['z','q','t','u','v']

field_ranges = {}
field_ranges['u']=(-24,30)
field_ranges['u']=(-23,26)
field_ranges['z']=(0,2000)
field_ranges['t']=(235,317)
field_ranges['q']=(0.0001,0.0222)

In [3]:
data=ds['t'].clip(min=field_ranges['t'][0],max=field_ranges['t'][1])
# del data

In [4]:
del data

In [14]:
import os
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

dpi=100

output_dir = "maps"
fields = ['z','t','u']
nstep=2

# Plot each frame with consistent colorbar
for field in fields:
    if field not in ds:
        continue

    vmin, vmax = field_ranges[field]
    data_field = ds[field]

    for step_index in range(nstep):
    # for step_index in range(data_field.sizes['step']):
        # Optionally: loop over levels if needed, here just one level (index 0)
        
        data=ds[field].isel(step=step_index, isobaricInhPa=0).clip(min=field_ranges[field][0],max=field_ranges[field][1])
        # data = data_field.isel(step=step_index, isobaricInhPa=0)

        fig = plt.figure(figsize=(8, 4), dpi=dpi)
        ax = plt.axes(projection=ccrs.PlateCarree())
        if step_index==0:
            ax.set_extent([-180, 180, 90, -90], crs=ccrs.PlateCarree())
        ax.coastlines()

        im = ax.pcolormesh(
            ds.longitude,
            ds.latitude,
            data,
            cmap="viridis",    # or choose another like 'plasma', 'coolwarm', etc.
            vmin=vmin,
            vmax=vmax,
            transform=ccrs.PlateCarree()
        )

        ax.set_global()
        ax.axis('off')

        filename = os.path.join(output_dir, f"{field}_{step_index:02d}.png")
        plt.savefig(filename, bbox_inches='tight', pad_inches=0, transparent=True)
        plt.close(fig)
        del data


In [12]:
grib_file = "20250427000000-0h-oper-fc.grib2"
ds0 = xr.open_dataset(grib_file, engine="cfgrib",backend_kwargs={
    "filter_by_keys": {"typeOfLevel": "isobaricInhPa"}
})

In [13]:
ds0

<xarray.Dataset> Size: 324MB
Dimensions:        (isobaricInhPa: 13, latitude: 721, longitude: 1440)
Coordinates:
    time           datetime64[ns] 8B ...
    step           timedelta64[ns] 8B ...
  * isobaricInhPa  (isobaricInhPa) float64 104B 1e+03 925.0 850.0 ... 100.0 50.0
  * latitude       (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude      (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
    valid_time     datetime64[ns] 8B ...
Data variables:
    z              (isobaricInhPa, latitude, longitude) float32 54MB ...
    q              (isobaricInhPa, latitude, longitude) float32 54MB ...
    u              (isobaricInhPa, latitude, longitude) float32 54MB ...
    w              (isobaricInhPa, latitude, longitude) float32 54MB ...
    t              (isobaricInhPa, latitude, longitude) float32 54MB ...
    v              (isobaricInhPa, latitude, longitude) float32 54MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-04-27T20:00 GRIB to CDM+CF via cfgrib-0.9.1...

In [21]:
import xarray as xr

grib_file = "pangu_20250422_0000_24.grib"  #"panguweather_20250427_0000_24.grib"
ds = xr.open_dataset(grib_file, engine="cfgrib")

output_dir = "maps"
fields = ['z','q','t','u','v']

field_ranges = {}
field_ranges['u']=(-24,30)
field_ranges['u']=(-23,26)
field_ranges['z']=(0,2000)
field_ranges['t']=(235,317)
field_ranges['q']=(0.0001,0.0222)

import os
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

dpi=100

output_dir = "maps"
fields = ['z','t','u']
nstep=2

# Plot each frame with consistent colorbar
for field in fields:
    if field not in ds:
        continue

    vmin, vmax = field_ranges[field]
    data_field = ds[field]

    for step_index in range(nstep):
    # for step_index in range(data_field.sizes['step']):
        # Optionally: loop over levels if needed, here just one level (index 0)
        
        data=ds[field].isel(step=step_index, isobaricInhPa=0).clip(min=field_ranges[field][0],max=field_ranges[field][1])
        # data = data_field.isel(step=step_index, isobaricInhPa=0)

        fig = plt.figure(figsize=(8, 4), dpi=dpi)
        ax = plt.axes(projection=ccrs.PlateCarree())
        # if step_index>0:
        #     ax.set_extent(crs=ccrs.PlateCarree(180))
        ax.coastlines()

        im = ax.pcolormesh(
            ds.longitude,
            ds.latitude,
            data,
            cmap="viridis",    # or choose another like 'plasma', 'coolwarm', etc.
            vmin=vmin,
            vmax=vmax,
            transform=ccrs.PlateCarree()
        )

        ax.set_global()
        ax.axis('off')

        filename = os.path.join(output_dir, f"{field}_{step_index:02d}.png")
        plt.savefig(filename, bbox_inches='tight', pad_inches=0, transparent=True)
        plt.close(fig)
        del data


skipping variable: paramId==167 shortName='t2m'
Traceback (most recent call last):
  File "/home/sylvain/Documents/training/ecmwf/.venv/lib/python3.11/site-packages/cfgrib/dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/sylvain/Documents/training/ecmwf/.venv/lib/python3.11/site-packages/cfgrib/dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='heightAboveGround' value=Variable(dimensions=(), data=np.float64(10.0)) new_value=Variable(dimensions=(), data=np.float64(2.0))
